In [1]:
!pip install -q transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.9 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.3.3.83 which is incompatible.
torch 2.5.1+cu124 requires nvidia-curand-cu12==10.3.5.147; platform_system == "Linux" and platfo

In [2]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HuggingfaceAPI")

login(token=hf_token)


In [3]:
from transformers import MarianTokenizer, MarianMTModel

model_name = "Helsinki-NLP/opus-mt-ur-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
# Save them before training (optional but good for consistency)
model.save_pretrained("./results")
tokenizer.save_pretrained("./results")


2025-05-12 19:07:13.247691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747076833.405165      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747076833.450222      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/848k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/816k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62024]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.json',
 './results/source.spm',
 './results/target.spm',
 './results/added_tokens.json')

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Your loaded text files
with open("/kaggle/input/parallel-corpus-for-english-urdu-language/Dataset/english-corpus.txt", "r", encoding="utf-8") as f:
    english_lines = f.read().splitlines()

with open("/kaggle/input/parallel-corpus-for-english-urdu-language/Dataset/urdu-corpus.txt", "r", encoding="utf-8") as f:
    urdu_lines = f.read().splitlines()

# Filter misaligned lines
pairs = [(u, e) for u, e in zip(urdu_lines, english_lines) if u.strip() and e.strip()]
urdu_lines, english_lines = zip(*pairs)

df = pd.DataFrame({"translation": [{"ur": u, "en": e} for u, e in zip(urdu_lines, english_lines)]})

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)


model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

In [5]:
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict


In [6]:
max_length = 128

def preprocess(batch):
    src_texts = [example["en"] for example in batch["translation"]]
    tgt_texts = [example["ur"] for example in batch["translation"]]

    model_inputs = tokenizer(src_texts, truncation=True, padding="max_length", max_length=max_length)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(tgt_texts, truncation=True, padding="max_length", max_length=max_length)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



train_ds = train_ds.map(preprocess, batched=True)
val_ds = val_ds.map(preprocess, batched=True)


Map:   0%|          | 0/22071 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2453 [00:00<?, ? examples/s]

In [7]:
# View a sample from the processed training dataset
sample = train_ds[1000]

# Decode inputs and labels to check correctness
print("Input (Urdu):", tokenizer.decode(sample["input_ids"], skip_special_tokens=True))
print("Label (English):", tokenizer.decode(sample["labels"], skip_special_tokens=True))


Input (Urdu): but it does not take much time
Label (English): لیکن یہ زیادہ وقت نہیں لیتا ہے


In [8]:
!pip install -U transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 70.5 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [9]:
pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install sacrebleu rouge-score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ba0072cade29636de00a2d6e5bc5b92292ada0779fd54983a953e41fc9656533
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [11]:
import evaluate
import numpy as np
from nltk.tokenize import sent_tokenize

bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # Sentence tokenize for ROUGE
    preds = ["\n".join(sent_tokenize(p)) for p in preds]
    labels = ["\n".join(sent_tokenize(l)) for l in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    bleu_result = bleu.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return {
        "bleu": round(bleu_result["score"], 4),
        "rouge1": round(rouge_result["rouge1"], 4),
        "rouge2": round(rouge_result["rouge2"], 4),
        "rougeL": round(rouge_result["rougeL"], 4),
    }


In [12]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=100,  # Log every 10 steps
    save_total_limit=1,
    predict_with_generate=True,
    report_to="none",  # Avoid integration errors (e.g., wandb)
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipykernel_31/3956891639.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,0.502000
200,0.330600
300,0.303400
400,0.281800
500,0.270200
600,0.259600
700,0.251600
800,0.241200
900,0.232100
1000,0.229600


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=2760, training_loss=0.2291876084562661, metrics={'train_runtime': 1406.2327, 'train_samples_per_second': 62.781, 'train_steps_per_second': 1.963, 'total_flos': 2992683249303552.0, 'train_loss': 0.2291876084562661, 'epoch': 4.0})

In [13]:
trainer.save_model("./fine-tuned-en-ur")
tokenizer.save_pretrained("./fine-tuned-en-ur")


('./fine-tuned-en-ur/tokenizer_config.json',
 './fine-tuned-en-ur/special_tokens_map.json',
 './fine-tuned-en-ur/vocab.json',
 './fine-tuned-en-ur/source.spm',
 './fine-tuned-en-ur/target.spm',
 './fine-tuned-en-ur/added_tokens.json')

In [14]:
from evaluate import load
from tqdm import tqdm
import torch

# Load metrics
bleu = load("sacrebleu")
rouge = load("rouge")

# Get source and reference texts
source_texts = [ex["translation"]["ur"] for ex in val_ds]
target_texts = [ex["translation"]["en"] for ex in val_ds]

# Generate predictions
def generate_predictions(model, tokenizer, sources, batch_size=16, max_length=128):
    model.eval()
    predictions = []
    for i in tqdm(range(0, len(sources), batch_size)):
        batch_texts = sources[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(model.device)
        with torch.no_grad():
            output_ids = model.generate(**inputs, max_length=max_length)
        batch_preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        predictions.extend(batch_preds)
    return predictions

# Run generation
preds = generate_predictions(model, tokenizer, source_texts)

# Post-process
preds = [p.strip() for p in preds]
refs = [[t.strip()] for t in target_texts]  # BLEU expects list of references per prediction

# Compute metrics
bleu_score = bleu.compute(predictions=preds, references=refs)
rouge_score = rouge.compute(predictions=preds, references=[r[0] for r in refs])

print("\n=== Evaluation Metrics ===")
print(f"BLEU score: {bleu_score['score']:.2f}")
print(f"ROUGE-L: {rouge_score['rougeL']:.2f}")


100%|██████████| 154/154 [00:54<00:00,  2.82it/s]



=== Evaluation Metrics ===
BLEU score: 1.11
ROUGE-L: 0.05


In [15]:
from transformers import MarianMTModel, MarianTokenizer, pipeline

model_path = "/kaggle/working/fine-tuned-en-ur"  # or replace with the actual path to the fine-tuned model
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)

translation_pipeline = pipeline("translation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [16]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 32.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 102.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.2 MB/s eta 0:00:00


In [17]:
import gradio as gr

def translate_text(text):
    # Use the translation_pipeline configured for English -> Urdu
    result = translation_pipeline(text)
    return result[0]['translation_text']

demo = gr.Interface(
    fn=translate_text,
    inputs=gr.Textbox(lines=5, placeholder="Enter English text..."),
    outputs="text",
    title="English to Urdu Translator",
    description="This model translates English text into Urdu using a fine-tuned translation pipeline."
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://deaaf545c5fb85715d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
